## Data conversion for CartoSeq

This notebook illustrates the codes and process for generating the joined dataset for CartoSeq, SeqScope mapping application.

We neeed to join the feature info and spatial coordinates to the count matrix.  The output will have global spatial coordinates and counts in one table. See the documentation for the coordinate conversion:

https://docs.google.com/document/d/11AkH8Qd4ux-wm3ib76c1kD3r7UXzVD1dJw_ES-jBKD4

The input files are three lane-wise combined dataset(barcodes.tsv.gz, matrix.mtx.gz, features.tsv.gz) and manifest.tsv for the whole dataset. Output file is gzipped tsv. 

In [1]:
import pandas as pd

In [2]:
# inputs (per lane dataset)
barcode_path  = "/Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/barcodes.tsv.gz"
matrix_path   = "/Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/matrix.mtx.gz"
feature_path  = "/Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/features.tsv.gz"
manifest_path = "/Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/manifest.tsv"

# output 
output_path   = "/Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/full_genes_coords.tsv.gz"

In [13]:
def lc2gc(df, grid_width, grid_height, manifest):
    ''' 
    add global coordinates to barcode dataframe 
    '''
     
    p = df['tile'] // 1000                 # first digit of tile number 
    q = (df['tile']  - p  * 1000) // 100   # second digit
    k = df['tile'] - p * 1000 - q * 100    # last two digits
    df['m'] = (p - 1) * 16 + k             # row number of tile from the bottom
    df['n'] = 2 * (2- df['lane']) + q      # col number of tile from the left
    
    # add xmin, ymin of a tile from manifest
    df = df.assign(
        tile_id=df.lane.map(str) + "_" + df.tile.map(str)
    ).set_index('tile_id')
    df = df.join(manifest[['xmin', 'ymin']])
    
    # global coords
    df['x'] = df.x_ + grid_width  * (df.m-1) - df.ymin  # note that xmin, ymin came from manifest
    df['y'] = df.y_ + grid_height * (df.n-1) - df.xmin  # where x, y flipped
    
    df = df[['barcode_id',  'x', 'y']].set_index('barcode_id')
    return df


def create_full_gene_pointdata(
    barcode_path, 
    matrix_path, 
    feature_path, 
    manifest_path, 
    output_path):
    
    ''' 
    read three tables and the matching manifest; 
    join and add global coordinates; 
    then save the joined output as tsv.gz
    ''' 
    
    # read barcodes
    df_b = pd.read_csv(
        barcode_path, 
        sep="\t",
        names=[
            'barcode', 'barcode_id', 'col1', 
            'lane', 'tile',
            'y_', 'x_', 'counts'],
        dtype = {
            'barcode': str,
            'barcode_id': 'int32',
            'y_': 'int32',
            'x_': 'int32',
        }
    )
    
    # read manifest
    manifest = pd.read_csv(manifest_path, sep="\t").set_index('id')
    manifest['width'] = manifest['ymax'] - manifest['ymin']
    manifest['height'] = manifest['xmax'] - manifest['xmin']
    grid_width = manifest.width.max()    # tile grid width
    grid_height = manifest.height.max()  # tile grid height
    
    # add global coordinates
    df_b = lc2gc(df_b, grid_width, grid_height, manifest)
    
    # read matrix 
    df_m = pd.read_csv(matrix_path, sep=" ", skiprows=3,
        names = [
            'gene_id', 'barcode_id', 
            'Gene', 'GeneFull', 
            'VelocytoSpliced', 'VelocytoUnspliced', 
            'VelocytoAmbiguous'],
        dtype= {
            'gene_id': 'int32',
            'barcode_id': 'int32',
            'Gene': 'int8',
            'GeneFull': 'int8',
            'VelocytoSpliced': 'int8', 
            'VelocytoUnspliced': 'int8',
            'VelocytoAmbiguous': 'int8'
        }
    )
    
    # read gene table
    df_g = pd.read_csv(feature_path, sep="\t",
        names = ['name', 'gene_name', 'gene_id',  'counts'],
        dtype = {
            'name': str,
            'gene_name': str,
            'gene_id': 'int32',
            'counts': str
        }
    ).set_index('gene_id')[['gene_name']]
    
    # join coords and gene_name to matrix
    df_merged = (df_m
        .merge(df_b, on='barcode_id')
        .merge(df_g, on='gene_id')
    ).drop(['gene_id', 'barcode_id'], axis=1)
    df_merged['cnt_total'] =  \
        df_merged['Gene'] +\
        df_merged['GeneFull'] +\
        df_merged['VelocytoSpliced'] +\
        df_merged['VelocytoUnspliced'] +\
        df_merged['VelocytoAmbiguous']
    
    
    df_merged.to_csv(
        output_path, 
        sep="\t", 
        index=False,
        compression='gzip'
    )
    print(f"Table with {len(df_merged)} rows saved at {output_path}")
    return df_merged 


In [14]:
df = create_full_gene_pointdata(barcode_path, matrix_path, feature_path, manifest_path, output_path)
df.head()

Table with 5438714 rows saved at /Users/yonghah/data/seqscope/hd31-hmkyv-mouse-1/full_genes_coords.tsv.gz


,Gene,GeneFull,VelocytoSpliced,VelocytoUnspliced,VelocytoAmbiguous,x,y,gene_name,cnt_total
0,0,1,0,1,0,71523,51953,Rabgap1l,2
1,0,1,0,0,0,2194285,50411,Rabgap1l,1
2,0,1,0,1,0,2750288,76359,Rabgap1l,2
3,0,1,0,1,0,2738906,51269,Rabgap1l,2
4,0,1,0,1,0,2100944,53722,Rabgap1l,2
